## Simulate SIR Data
Created: June 26, 2020 Olivia and Harrison\
Updated: Feb. 26, 2021 Olivia, Matthew and Harrison


### SIR Stochastic Model

Epidemics are intrinsically stochastic systems, therefore, it is natural to attempt to capture aspects of the random elements of an epidemic using a stochastic (that is, a Monte Carlo) model. Other models attempt to estimate the *mean* behavior of epidemics. Such models should be consistent with and, ideally, follow from the stochastic model as we show below.

__Variables__

$$
\begin{aligned}
    S(t) &= \text{mean number of susceptible persons at time } t,\\
    I(t) &= \text{mean number of infected persons at time } t,\\
    R(t) &= \text{mean number of recovered (removed) persons at time } t,\\
    m    &= \text{number of susceptible persons at time } t,\\
    n    &= \text{number of infected persons at time } t,\\
\end{aligned}
$$
__Parameters__
$$
\begin{aligned}
    \alpha &= \text{recovery rate (so } 1/\alpha \text{ is the mean infectious period)}\\
    \beta  &= \text{transmission rate per infected person}
\end{aligned}
$$
__Stochastic Model__

A stochastic model deals with transition probabilities over a time interval $h$ that is so short that only one of $K$ possible things that can happen in the interval $(t, t + h)$ occurs. The key idea is to assume a state at time $t + h$ and relate it probabilistically to the possible states at time $t$. The counts $m$ and $n$ define the state of the stochastic system. In this model we assume that only three things can happen: 1) an infection occurs or 2) a recovery of removal from the infected category occurs or 3) nothing happens. The transition probabilities are:

$$
 \begin{array}
    & 
    \text{state at time } t \quad & 
    \text{state at time } t + h \quad &
    \text{transition probability }\\
    \text{infection } & m + 1, n - 1 & m, n & p_1 = \beta (m + 1) (n - 1) \, h + {\cal O}(h^2)\\
    \text{removal } & m, n + 1 & m, n & p_2 = \alpha (n + 1) \, h + {\cal O}(h^2)\\
   \text{none } & m, n & m, n & p_3 = 1 - (\beta m n + \alpha n ) \, h + {\cal O}(h^2).
\end{array}
$$ 

Therefore, the probability to arrive at state $(m, n)$ at time $t + h$ is given by
$$
\begin{aligned}
    p_{m, n}(t + h) & = p_1 p_{m+1, n-1}(t) + p_2 p_{m, n+1}(t) + p_3 p_{m,n}(t).
\end{aligned}
$$

Writing $\Delta p_{m,n} = p_{m,n}(t + h) - p_{m,n}(t)$, dividing by $h$, and letting $h \rightarrow 0$, yields the Kolmogorov equations for the SIR model,

$$


\begin{aligned}
\boxed{\frac{dp_{m, n}}{dt} = \beta \, (m+1)(n-1) \, p_{m+1, n-1}(t)
                + \alpha \, (n+1) \,  p_{m, n+1}(t) 
                - (\beta m n + \alpha n) \, p_{m,n}(t),}
\end{aligned}
               
$$ 

the solution of which is the probability $p_{m,n}(t)$ that at time $t$ there are exactly $m$ susceptible and $n$ infected persons. Define the following means

$$
\begin{aligned}
    S(t) & \equiv \langle m \rangle = \sum_{m, n} m \, p_{m,n}(t), \\
    I(t) & \equiv \langle n \rangle = \sum_{m, n} n \, p_{m,n}(t),
\end{aligned}
$$

where the sum is over all pairs of integers $(0 \leq m \leq N, 0 \leq n \leq N)$ subject to the constraint $m + n \leq N$,
where $N$ is the size of the population, which is assumed to be fixed. The cardinality of the set $\{ (m, n) \}$ is $(N + 1) (N + 2)/2$. The mean function $S(t)$ satisfies the ordinary differential equation (ODE) 

$$
\begin{aligned}
\frac{dS}{dt} & = \sum_{m, n} m \, \beta \, (m+1)(n-1) \, p_{m+1, n-1}\\
              & + \sum_{m, n} m \, \alpha \, (n+1) \,  p_{m, n+1} \\
              &  - \sum_{m, n} m \, (\beta m n + \alpha n) \, p_{m,n}.
\end{aligned}
$$

We now try to write the above in terms of $S(t)$ and $I(t)$. To that end, change the indices in each sum so that the probabilities can be expressed as $p_{s, i}$. We can do this with the substitutions $m \rightarrow s - 1$ and $n \rightarrow i + 1$ in the first sum, $n \rightarrow i - 1$ in the second sum, and $m \rightarrow s$ and $n \rightarrow i$ in the third sum. (We use $s(t)$ and $i(t)$ to remind us that these integers are the counts of susceptible and infected persons at time $t$, which, of course, are not necessarily equal to the associated means $S(t)$ and $I(t)$, respectively.) The ODE can now be written as

$$
\begin{aligned}
  \frac{dS}{dt} & = \sum_{s, i} \left[ \beta \, (s - 1) \, s i  
                + \alpha \, s i 
                - (\beta \, s^2 i + \alpha \, s i) \right] p_{s, i}, \\
                & = -\beta \sum_{s, i} s i  \,  p_{s, i}, \\
                & = -\beta \, \langle s i \rangle = -\beta \, Q_{11},
\end{aligned}
$$

where we have defined the double moments $Q_{ab} \equiv \langle s^a i^b \rangle =  \sum_{s, i} s^a(t) i^b(t) \, p_{s, i}$.
A similar derivation yields

$$
\begin{aligned}
\frac{dI}{dt} & = -\alpha \, I + \beta \, Q_{11},
\end{aligned}
$$

noting that $I(t) = Q_{01}(t)$. 


If the correlation between $s(t)$ and $i(t)$ can be neglected, we may assume $\langle s(t) i(t) \rangle \approx \langle s(t) \rangle  \langle  i(t) \rangle = S(t) I(t)$. With this approximation, we arrive at the commonly used 2-parameter deterministic equations of the SIR model,

$$
\begin{aligned}
    \frac{dS}{dt} & = - \beta \, S I, \\
    \frac{dI}{dt} & = - \alpha I + \beta \, S I  .
\end{aligned}
$$

In [1]:
import os, sys
import numpy as np
import joblib as jb  # for saving Python objects

### Generate an epidemic

  1. $T$ times at which to report observations (counts s and i)
  1. $s_0$, $i_0$, $r_0$ initial counts of susceptibles, infected, and recovered.
  1. $t_\min, t_\max$ minimum and maximum observation times. Note $T_0 \equiv T[0] = t_\min$.
  1. $\alpha, \beta$ model parameters.
  1. $maxiter$ maximum number of micro-steps just to make sure we don't get stuck in a loop.

In [2]:
def generateTarget1(T, 
             s0, i0, r0, 
             tmin, tmax, 
             alpha, beta, 
             maxiter=500000):
    
    # initialize lists
    s  = [s0]
    i  = [i0]
    r  = [r0]
   
    st = s0
    it = i0
    rt = r0
   
    # get first observation reporting time beyond tmin.
    j  = 1
    Treport = T[j]
     
    # start time
    t  = tmin
    ii = 0
    #Ttest = [0];
    while (t < tmax) and (ii < maxiter): # might want to change to t<=tmax!!!!
        ii += 1
    
        # generate time to next event
        p1   = beta * it * st
        p2   = alpha* it
        psum = p1 + p2
        if psum <= 0: break

        t += np.random.exponential(1.0/psum)

        # Choose event
        pr = [p1/psum, p2/psum]
        k  = np.random.choice([1, 2], p=pr)
        
        i_new = 0
        r_new = 0  
        if k == 1:
            i_new = 1  # an infection event
        else:
            r_new = 1  # a recovery or removal event

        # save current state before update
        st_prev = st
        it_prev = it
        rt_prev = rt
        
        # update state
        st = st - i_new
        it = it + i_new - r_new
        rt = rt + r_new
        
        # if time exceeds next observation reporting time, 
        # return data of previous state
        if t > Treport:  # Should this be t>= Treport??? --No!
            s.append(st_prev)
            i.append(it_prev)
            r.append(rt_prev)
            
            #Ttest.append(Treport)
            # get next reporting time
            j += 1
            if j < len(T):  # Should this be j<=len(T)???
                Treport = T[j]
            else:
                break
        #deprecated test code print statement remove at a later date.        
        #print('Treport', Treport,s[-1],i[-1],r[-1],np.shape(s),np.shape(i),np.shape(r))    
        # if time exceeds last reporting time, we're done!
        if t > T[-1]: 
            break
     
    # convert python lists to numpy arrays
    s = np.array(s)
    i = np.array(i)
    r = np.array(r)
    
    return (s, i, r)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b323b509-75b9-423f-bb04-ac84a2ad5053' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>